In [ ]:
import intake

intake.output_notebook()

In [ ]:
intake.gui

This notebook demonstrates the new GUI implemented in [panel](http://panel.pyviz.org) rather than ipywidgets. As an example, I'll use the image catalog available at: https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml

In [ ]:
cat = intake.open_catalog('https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml')

In [ ]:
intake.gui.browser.cat.add(cat)

In [ ]:
intake.gui.browser.source.select('UCMerced_LandUse_by_image')

In [ ]:
intake.gui.item(id=3, landuse='airplane').plot.rgb(x='x', y='y', bands='channel', width=350)